## HR chatbot by Superlinked

I'm following a [tutorial](https://github.com/superlinked/superlinked/blob/main/notebook/rag_hr_knowledgebase.ipynb) created by Superlinked but making some changes: 
* Use ChatGPT instead of LLama2

This guide implies that you can deploy and run Llama2 locally (which I'm not doing and haven't tried).
* There is more information available on Huggingface.co ([Llama 2 is here - get it on Hugging Face](https://huggingface.co/blog/llama2)); and 
* on Meta's website ([5 Steps to Getting Started with Llama 2](https://ai.meta.com/blog/5-steps-to-getting-started-with-llama-2/))

### Installation

In [ ]:
%pip install superlinked==6.8.0
%pip install -U jupyter ipywidgets
%pip install -U torch transformers accelerate

**Explanation of modules**

* `ipywidgets`: Enables the use of interactive HTML widgets (e.g. sliders) within your Jupyter notebooks.
* `torch`: This is the package for PyTorch, an open-source machine learning library used for applications such as computer vision and natural language processing. PyTorch provides two high-level features: 
    * Tensor computation (like NumPy) with strong GPU acceleration.
    * Deep neural networks built on a tape-based autograd system.
* `transformers`: This is a package from Hugging Face that provides state-of-the-art machine learning models, particularly for natural language processing (NLP) tasks. It includes implementations of various transformer models like BERT, GPT, and T5.
* `accelerate`: This package is also from Hugging Face. It is designed to help with the efficient training of models across multiple GPUs, TPUs, or other hardware accelerators. It simplifies distributed training and optimization for large-scale models.

I think that most of these libraries are needed for Llama2 to work but since I'm using ChatGPT we probably don't need them. 

### Imports 

In [42]:
from datetime import timedelta, datetime

from dotenv import load_dotenv
from openai import OpenAI
import os 

import pandas as pd
import requests
import torch
from transformers import AutoTokenizer, pipeline

from superlinked.framework.common.dag.context import CONTEXT_COMMON, CONTEXT_COMMON_NOW
from superlinked.framework.common.dag.period_time import PeriodTime
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import String, Timestamp
from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.parser.dataframe_parser import DataFrameParser
from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.query.param import Param
from superlinked.framework.dsl.query.query import Query
from superlinked.framework.dsl.query.result import Result
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.space.text_similarity_space import (
    TextSimilaritySpace,
    chunk,
)
from superlinked.framework.dsl.space.recency_space import RecencySpace

### Constants

In [40]:
# pandas display settings
pd.set_option("display.max_colwidth", None)

# constants 
START_OF_2024_TS = int(datetime(2024, 1, 2).timestamp())
EXECUTOR_DATA = {CONTEXT_COMMON: {CONTEXT_COMMON_NOW: START_OF_2024_TS}}
TOP_N = 10

# environment variables (OpenAI key)
load_dotenv()

True

### Load data

In [4]:
# the new HR policy document
r_new = requests.get(
    "https://storage.googleapis.com/superlinked-notebook-hr-knowledgebase/hr_rag_knowledgebase.txt"
)
r_new.encoding = "utf-8-sig"
text_new = r_new.text.replace("\r\n", "\n").split("\n")
r_old = requests.get(
    "https://storage.googleapis.com/superlinked-notebook-hr-knowledgebase/hr_rag_old_text.txt"
)
r_old.encoding = "utf-8-sig"
text_old = r_old.text.replace("\r\n", "\n").split("\n")

In [23]:
text_df = pd.DataFrame(text_new + text_old, columns=["body"]).reset_index()
# add timestamps to differentiate the two sources
text_df["creation_date"] = [
    int(datetime(2024, 1, 1).timestamp())] * len(text_new) + [
    int(datetime(2023, 1, 1).timestamp())
] * len(text_old)
text_df.head()

,index,body,creation_date
0,0,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",1704067200
1,1,"This policy applies to all employees of <Company Name>, consultants, contractors and other third-party entities with access to <Company Name> production networks and system resources.",1704067200
2,2,"Background verification checks on <Company Name> personnel shall be carried out in accordance with relevant laws, regulations, and shall be proportional to the business requirements, the classification of the information to be accessed, and the perceived risks. Background screening shall include criminal history checks unless prohibited by local statute. All third-parties with technical privileged or administrative access to <Company Name> production systems or networks are subject to a background check or requirement to provide evidence of an acceptable background, based on their level of access and the perceived risk to <Company Name>.",1704067200
3,3,"The skills and competence of employees and contractors shall be assessed by human resources staff and the hiring manager or his or her designees as part of the hiring process. Required skills and competencies shall be listed in job descriptions and requisitions, and/or aligned with the responsibilities outlined in the Roles and Responsibilities Policy. Competency evaluations may include reference checks, education and certification verifications, technical testing, and interviews.",1704067200
4,4,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",1704067200


### Set up Superlinked for Retrieval

In [6]:
# typed schema to describe our inputs
@schema
class ParagraphSchema:
    body: String
    created_at: Timestamp
    id: IdField

We'll map the column names to reflect our DataFrame structure further down below (`index` not `id` and `creation_date` not `created_at`).

In [7]:
paragraph = ParagraphSchema()

In [ ]:
# relevance space will encode our knowledge base utilizing chunking to control the granularity of information
relevance_space = TextSimilaritySpace(
    text=chunk(paragraph.body, chunk_size=100, chunk_overlap=20),
    model="sentence-transformers/all-mpnet-base-v2",
)
# recency has a period time to differentiate between the document created at the beginning of this year and last year
recency_space = RecencySpace(
    timestamp=paragraph.created_at,
    period_time_list=[PeriodTime(timedelta(days=300))],
    negative_filter=-0.25,
)

In [9]:
paragraph_index = Index([relevance_space, recency_space])
paragraph_parser = DataFrameParser(
    paragraph, mapping={paragraph.id: "index", paragraph.created_at: "creation_date"}
)
source: InMemorySource = InMemorySource(paragraph, parser=paragraph_parser)
executor = InMemoryExecutor(
    sources=[source], indices=[paragraph_index], context_data=EXECUTOR_DATA
)
app = executor.run()
source.put([text_df])

In [10]:
# our simple query will make a search term possible, and gives us the opportunity to weight input aspects (relevance and recency against each other)
knowledgebase_query = (
    Query(
        paragraph_index,
        weights={
            relevance_space: Param("relevance_weight"),
            recency_space: Param("recency_weight"),
        },
    )
    .find(paragraph)
    .similar(relevance_space.text, Param("search_query"))
    .limit(Param("limit"))
)

### Perform retrieval

In [11]:
def present_result(result: Result):
    """A small helper function to present our query results"""
    df = result.to_pandas()
    df["year_created"] = [datetime.fromtimestamp(ts).year for ts in df["created_at"]]
    return df.drop("created_at", axis=1)

In [12]:
initial_query_text: str = "What should management monitor?"

First, let's do a simple retrieval based only on text similarity. We can see that the recency weight is set to 0, so the document creation dates are not affecting our results.

In [24]:
only_relevance_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0,
    search_query=initial_query_text,
    limit=TOP_N,
)

present_result(only_relevance_result)

,body,id,year_created
0,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",6,2024
1,"Management is tasked with ensuring that information security policies and procedures undergo an bi-annual review, are disseminated and accessible, and that employees and contractors adhere to them throughout their tenure or contract period. The yearly policy assessment must encompass a scrutiny of any interconnected or referenced protocols, standards, or directives.",16,2023
2,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",7,2024
3,"Management must guarantee that information security obligations are conveyed to individuals, either through written job outlines, policies, or alternative documented means that are regularly updated and managed accurately. Compliance with information security protocols and the fulfillment of related duties should form part of the performance evaluation process in all cases.",18,2023
4,"Management should factor in undue pressures and potential fraud scenarios when devising incentives and delineating roles, duties, and authorizations.",17,2023
5,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",8,2024
6,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0,2024
7,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",4,2024
8,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",11,2024
9,All <Company Name> employees and third-parties with administrative or privileged technical access to <Company Name> production systems and networks shall complete security awareness training at the time of hire and annually thereafter. Management shall monitor training completion and shall take appropriate steps to ensure compliance with this policy. Employees and contractors shall be aware of relevant information security and data privacy policies and procedures. The company shall ensure that personnel receive security and data privacy training appropriate to their role and data handling responsibilities.,9,2024


Let's observe the difference between elements with ids 16 and 6, respectively! These documents essentially say the same thing, except that the old document prescribes a **bi-annual**, while the fresh one an **annual** review on policies and procedures. Let's see what happens if we upweight recency.

In [25]:
mild_recency_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.15,
    search_query=initial_query_text,
    limit=TOP_N,
)

present_result(mild_recency_result)

,body,id,year_created
0,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",6,2024
1,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",7,2024
2,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",8,2024
3,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0,2024
4,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",4,2024
5,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",11,2024
6,All <Company Name> employees and third-parties with administrative or privileged technical access to <Company Name> production systems and networks shall complete security awareness training at the time of hire and annually thereafter. Management shall monitor training completion and shall take appropriate steps to ensure compliance with this policy. Employees and contractors shall be aware of relevant information security and data privacy policies and procedures. The company shall ensure that personnel receive security and data privacy training appropriate to their role and data handling responsibilities.,9,2024
7,"In order to maintain a robust level of security awareness, the company will provide security-related updates and communications to company personnel on an on-going basis through multiple communication channels as needed.",10,2024
8,"Management is tasked with ensuring that information security policies and procedures undergo an bi-annual review, are disseminated and accessible, and that employees and contractors adhere to them throughout their tenure or contract period. The yearly policy assessment must encompass a scrutiny of any interconnected or referenced protocols, standards, or directives.",16,2023
9,"Background verification checks on <Company Name> personnel shall be carried out in accordance with relevant laws, regulations, and shall be proportional to the business requirements, the classification of the information to be accessed, and the perceived risks. Background screening shall include criminal history checks unless prohibited by local statute. All third-parties with technical privileged or administrative access to <Company Name> production systems or networks are subject to a background check or requirement to provide evidence of an acceptable background, based on their level of access and the perceived risk to <Company Name>.",2,2024


Now, document the relative position of element 6 compared to 16 improved as it received a score boost for it's recency. We can see the same dynamics here between documents with IDs 7 and 18 - doc 7 softened the requirement to "wherever applicable" from "in all cases". It is a useful feature that documents stating essentially the same can be differentiated through their freshness - let's boost that and see if we can have only recent, not outdated, information in our results.

In [26]:
normal_recency_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.25,
    search_query=initial_query_text,
    limit=TOP_N,
)

norm_recency_result_df = present_result(normal_recency_result)
norm_recency_result_df

,body,id,year_created
0,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",6,2024
1,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",7,2024
2,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",8,2024
3,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0,2024
4,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",4,2024
5,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",11,2024
6,All <Company Name> employees and third-parties with administrative or privileged technical access to <Company Name> production systems and networks shall complete security awareness training at the time of hire and annually thereafter. Management shall monitor training completion and shall take appropriate steps to ensure compliance with this policy. Employees and contractors shall be aware of relevant information security and data privacy policies and procedures. The company shall ensure that personnel receive security and data privacy training appropriate to their role and data handling responsibilities.,9,2024
7,"In order to maintain a robust level of security awareness, the company will provide security-related updates and communications to company personnel on an on-going basis through multiple communication channels as needed.",10,2024
8,"Background verification checks on <Company Name> personnel shall be carried out in accordance with relevant laws, regulations, and shall be proportional to the business requirements, the classification of the information to be accessed, and the perceived risks. Background screening shall include criminal history checks unless prohibited by local statute. All third-parties with technical privileged or administrative access to <Company Name> production systems or networks are subject to a background check or requirement to provide evidence of an acceptable background, based on their level of access and the perceived risk to <Company Name>.",2,2024
9,"Company policies and information security roles and responsibilities shall be communicated to employees and third-parties at the time of hire or engagement, and employees and contractors are required to formally acknowledge their understanding and acceptance of their security responsibilities. Employees and third-parties with access to company or customer information shall sign an appropriate non-disclosure, confidentiality, and appropriate code-of-conduct agreements. Contractual agreements shall state responsibilities for information security as needed. Employees and relevant third-parties shall follow all <Company Name> information security policies.",5,2024


Yes, we can! Now that all results are fresh, the outdated counterparts were removed from the result set. This way we can supply a nice and clean context to the generation model.

It is also important to see that the same weights produce relevant results for queries about maternity leave—a topic only discussed in the older HR document. It does not get overruled by recent but not relevant documents.

In [27]:
normal_recency_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.25,
    search_query="What are the companies terms for maternal leave?",
    limit=TOP_N,
)

maternity_result = present_result(normal_recency_result)
maternity_result

,body,id,year_created
0,[Company Name] is committed to supporting employees during significant life events such as the birth or adoption of a child. Our Maternity Leave Policy aims to provide eligible employees with the necessary time off to bond with their newborn or newly adopted child and to ensure a smooth transition back to work.,22,2023
1,"Following childbirth or adoption, employees are entitled to [XX weeks/months] of paid maternity leave to recover from childbirth, bond with their newborn or adopted child, and adjust to their new family dynamic. This period of leave is intended to promote the physical and emotional well-being of both the parent and child during the crucial early stages of development.",26,2023
2,"[Company Name] is committed to maintaining the confidentiality of employees' maternity leave and pregnancy-related information in accordance with applicable privacy laws. Additionally, [Company Name] prohibits discrimination or retaliation against employees for exercising their rights under this policy or applicable law.",30,2023
3,"Eligible employees are entitled to up to [XX weeks/months] of paid maternity leave, which includes both prenatal and postnatal leave. Additionally, eligible employees may also take up to [XX weeks/months] of unpaid maternity leave following the conclusion of their paid leave, as provided under the Family and Medical Leave Act (FMLA) and applicable state laws.",24,2023
4,"[Company Name] recognizes the importance of work-life balance and supports flexible work arrangements, including telecommuting and adjusted work schedules, to facilitate a smooth transition back to work for new parents. Employees returning from maternity leave are encouraged to discuss their individual needs and preferences with their supervisor and HR department to explore available options.",27,2023
5,"All full-time and part-time employees are eligible for maternity leave if they have been employed by [Company Name] for at least 12 months and have worked a minimum of 1,250 hours during the 12-month period preceding the commencement of leave. This policy applies equally to biological mothers, adoptive parents, and employees expecting the birth of a child through surrogacy.",23,2023
6,"At the conclusion of maternity leave, employees are expected to return to their regular duties and responsibilities. [Company Name] will make reasonable efforts to accommodate the transition back to work, including providing any necessary training or support to help employees reintegrate into the workplace successfully.",29,2023
7,This Maternity Leave Policy will be reviewed periodically by the HR department to ensure compliance with relevant laws and regulations and to assess its effectiveness in supporting the needs of employees and their families.,32,2023
8,"During maternity leave, eligible employees will continue to receive their regular salary and benefits in accordance with [Company Name]'s existing benefits policies. Employees may also continue their health insurance coverage during unpaid leave by electing to pay the required premiums as permitted under the Consolidated Omnibus Budget Reconciliation Act (COBRA).",28,2023
9,"Expectant mothers may commence their maternity leave [XX weeks/months] before the expected date of delivery to attend prenatal appointments, prepare for childbirth, and address any pregnancy-related complications. The timing of prenatal leave should be discussed and coordinated with the employee's supervisor and HR department.",25,2023


### Augmentation - formulate query for our LLM for generation

In [37]:
context_items_from_retrieval: int = 5
context_text: str = (
    "\n"
    + "\n".join(
        f for f in norm_recency_result_df["body"].iloc[:context_items_from_retrieval]
    )
    + "\n"
)

In [38]:
print(context_text)


Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.
Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.
Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.
To ensure that employees and contracto

### Generation - prompt the LLM for text generation to answer our query

In [43]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [44]:
ai_system_context = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
"""

In [45]:
def generate_response(context, query):
    """
    Generates a response using ChatGPT.

    Parameters:
    context (str): The context or conversation history.
    query (str): The user's question or message.

    Returns:
    str: The generated response from ChatGPT.
    """
    chat_completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": ai_system_context
            },
            {
                "role": "user",
                "content": f"Context: {context}\n\nQuestion: {query}",
            }
        ],
        temperature=0.7,
    )

    return chat_completion.choices[0].message.content

In [47]:
print(generate_response(context_text, initial_query_text))

Management should monitor the following:

1. **Annual Review of Policies and Procedures**: Ensure that information security policies and procedures are reviewed annually, including any linked or referenced procedures, standards, or guidelines.

2. **Distribution and Availability of Policies**: Confirm that information security policies and procedures are distributed to all relevant parties and are readily available for reference.

3. **Adherence to Policies and Procedures**: Verify that employees and contractors abide by the information security policies and procedures throughout their tenure.

4. **Communication of Responsibilities**: Ensure that information security responsibilities are clearly communicated to individuals through documented methods such as written job descriptions or policies.

5. **Updates and Maintenance**: Maintain accurate and up-to-date documentation of information security responsibilities.

6. **Performance Evaluation**: Incorporate compliance with information